### 네이버 뉴스 크롤링 최종
 - 섹션 추출 ~ sub 페이지 추출

#### (1) + (2) + (3)
(1) 섹션 메뉴와 섹션별 url 추출   
    - section, link     
    - section_menu_df  
(2) 섹션별 헤드라인(topic) 제목과 url 추출   
    - topic, url, section     
    - all_topic_df  
(3) 섹션 헤드라인별 sub 페이지 기사 내용 추출   
    - 언론사, 제목, 날짜/시간, 기사내용, 작성자     
    - paper, title, datetime, writing, writer     
    - all_sub_news_info_df  

In [1]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity="all"
from urllib.request import urlopen
import bs4
import pandas as pd
import requests

In [2]:
url = 'https://news.naver.com/'
html = urlopen(url)
# 파서 객체 생성
bs_obj = bs4.BeautifulSoup(html, 'html.parser')

#### (1) 네이버 뉴스 섹션 메뉴와 url 추출

In [3]:
ul = bs_obj.find('ul',{'class':'Nlnb_menu_list'})
lis = ul.findAll('li')

In [11]:
section = []
link = []

for li in lis:
    a_tag = li.find('a')
    section.append(a_tag.text)
    link.append(a_tag['href'])

section_menu_df = pd.DataFrame({'section':section, 'link':link})
section_menu_df

,section,link
0,언론사별,https://news.naver.com/?viewType=pc
1,정치,https://news.naver.com/main/main.naver?mode=LS...
2,경제,https://news.naver.com/main/main.naver?mode=LS...
3,사회,https://news.naver.com/main/main.naver?mode=LS...
4,생활/문화,https://news.naver.com/main/main.naver?mode=LS...
5,IT/과학,https://news.naver.com/main/main.naver?mode=LS...
6,세계,https://news.naver.com/main/main.naver?mode=LS...
7,랭킹,https://news.naver.com/main/ranking/popularDay...
8,신문보기,https://news.naver.com/newspaper/home?viewType=pc
9,오피니언,https://news.naver.com/opinion/home


#### (2) 섹션별 헤드라인(topic) 제목과 url 추출

In [5]:
def get_topic(url, section):
    # 접속 및 결과 파싱
    headers = {'User-Agent':'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/116.0.0.0 Safari/537.36'}
    res = requests.get(url, headers=headers)
    html = res.text
    bs_obj = bs4.BeautifulSoup(html, 'html.parser')

    # topic_list 추출
    topic_list = bs_obj.findAll('div',{'class':'sh_text'})

    # 제목과 링크 추출해서 리스트에 추가
    topic_title = []
    topic_link = []

    for topic in topic_list:
        try :
            a = topic.find('a')
            topic_title.append(a.text)
            topic_link.append(a['href'])
        except:
            print('오류 발생')
            # pass
            
    # 딕셔너리 만들어서 반환
    return ({'topic':topic_title, 'url':topic_link, 'section':section})

In [13]:
# 모든 섹션에 대한 모든 topic 데이터 프레임 생성
all_topic_df = pd.DataFrame({
    'topic':[],
    'url':[],
    'section':[]
})

for i in range(1, 7):
    df = pd.DataFrame(get_topic(section_menu_df['link'][i], section_menu_df['section'][i]))
    all_topic_df = pd.concat([all_topic_df, df], axis=0, ignore_index=True)
    # axis=0 : 행으로 결합
    # ignore_index=True : 기존 df의 인덱스 무시 (모든 행을 새로운 인덱스로 설정)

all_topic_df

,topic,url,section
0,"러 ""북한과 대북 유엔제재 논의 준비돼…안보리서 북한과 공조""",https://n.news.naver.com/mnews/article/055/000...,정치
1,김성태 “이재명 단식의 진정성 반 정도는 인정해야… 단식은 약자의 최후 수단”,https://n.news.naver.com/mnews/article/022/000...,정치
2,"김정은 ""4년만에 방러, 북러관계 전략적 중요성 뚜렷이 표현""",https://n.news.naver.com/mnews/article/025/000...,정치
3,"[속보] 북, 김정은-푸틴 회담 앞두고 탄도미사일 2발 발사",https://n.news.naver.com/mnews/article/081/000...,정치
4,"국힘, 뉴스타파 인용 보도 KBS·TBS 라디오 진행자 3명 고발",https://n.news.naver.com/mnews/article/028/000...,정치
5,“내년 9090억원 투입…전 세계에서 AI 잘 활용하는 나라 만들겠다”,https://n.news.naver.com/mnews/article/016/000...,정치
6,"尹대통령, 개각 발표 오후 3시로 연기…국방·문체·여가부 장관 교체",https://n.news.naver.com/mnews/article/008/000...,정치
7,"김기현, '오염수 방류 우려' 김윤아에 ""무개념 연예인""",https://n.news.naver.com/mnews/article/469/000...,정치
8,"한총리, 프라하서 체코 총리와 회담…""에너지·인프라 협력강화""",https://n.news.naver.com/mnews/article/001/001...,정치
9,"이재명 ""국방장관 교체는 꼬리자르기""…'단식 14일차' 장소 변경",https://n.news.naver.com/mnews/article/422/000...,정치


#### (3) 섹션 헤드라인별 sub 페이지 기사 내용 추출
- 언론사, 제목, 날짜/시간, 기사내용, 작성자

In [7]:
# sub 페이지 기사 내용 추출하는 함수
def get_sub_news_info(url):
    headers = {'User-Agent':'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/116.0.0.0 Safari/537.36'}
    # 빈 리스트 생성
    paper_list, title_list, datetime_list, writing_list, writer_list = [ [] for _ in range(5) ]
    # for _ in range(5) : 언더바(_) *변수 없이 반복만 수행)
    try :
        res = requests.get(url, headers=headers)
        html = res.text
        bs_obj = bs4.BeautifulSoup(html, 'html.parser')

        # 접속


        # 언론사, 기사제목, 작성 날짜 시간, 기사 내용, 작성자 리스트 생성
        paper_list.append(bs_obj.find('a', {'class':'media_end_head_top_logo'}).select_one('img')['title'])
        title_list.append(bs_obj.find('div', {'class' : 'media_end_head_title'}).select_one('span').text)
        datetime_list.append(bs_obj.find('span', {'class' : 'media_end_head_info_datestamp_time _ARTICLE_DATE_TIME'}).text)
        writing_list.append(bs_obj.select_one('#dic_area').text.replace('\n', ''))
        writer_list.append(bs_obj.find('span',{'class':'byline_s'}).text)
    except:
        print('오류 발생')
        # pass


    # 딕셔너리로 만들어서 반환
    sub_news_dict = {
        'paper':paper_list,
        'title':title_list,
        'datetime':datetime_list,
        'writing':writing_list,
        'writer':writer_list
        
    }
    return sub_news_dict

In [8]:
# 모든 sub 페이지 내용을 저장하는 데이터프레임 생성해서 최종 데이터프레임 생성
all_sub_news_info_df = pd.DataFrame({
    'paper': [],
    'title': [],
    'datetime': [],
    'writing': [],
    'writer': []
})

for url in all_topic_df['url']:
    df = pd.DataFrame(get_sub_news_info(url))
    all_sub_news_info_df = pd.concat([all_sub_news_info_df, df], axis=0, ignore_index=True)

all_sub_news_info_df

,paper,title,datetime,writing,writer
0,SBS,"러시아 방문한 김정은 영상 공개돼…""내일 푸틴과 회담""",2023.09.12. 오후 11:44,4년 5개월 만에 러시아를 방문한 김정은 북한 국무위원장의 모습이 공개됐습니다.러시...,윤나라 기자 invictus@sbs.co.kr
1,파이낸셜뉴스,진중권 “박지현 눈물은 ‘공천용’...이재명에 회복식? 뻔한 거 아니냐”,2023.09.13. 오전 9:30,지난 11일 오후 단식 12일째인 이재명 더불어민주당 대표의 단식농성장을 찾은 박지...,박상훈 기자 (sanghoon3197@fnnews.com)
2,뉴시스,"김기현, 교원단체와 '교권회복' 간담회…법안 처리 고삐",2023.09.13. 오전 12:01,"교권회복 4법·아동학대처벌특례법 개정 등 논의여야, 교육위 법안소위서 교권회복 4법...",정성원 기자(jungsw@newsis.com)
3,데일리안,"단식 중인 이재명 항의방문했다고…민주당, 태영호 징계안 제출하기로",2023.09.12. 오전 11:37,"지난 7일 ""北에서 온 쓰레기"" 발언 항의방문송기헌 ""태영호 행태 도저히 묵과할 수...",김은지 기자 (kimeunji@dailian.co.kr)
4,조선비즈,"김기현, 자우림 김윤아 저격…“개념 없는 개념연예인”",2023.09.12. 오후 5:36,국민의힘 김기현 대표가 12일 오후 서울 중구 한국관광공사에서 열린 사단법인 문화자...,오귀환 기자 ogi@chosunbiz.com
5,KBS,"이재명 “윤석열 정부, 국방장관 교체로 꼬리자르기 열중…특검으로 진상 규명”",2023.09.13. 오전 10:04,더불어민주당 이재명 대표는 이종섭 국방부 장관의 사의 표명과 관련해 “윤석열 정부는...,고은희 (ginger@kbs.co.kr)
6,파이낸셜뉴스,‘한중일 대 북중러’ 대결 구도 굳어지나? 연쇄회담 가능성,2023.09.13. 오전 11:22,- 한중일에 맞서 결집하는 북러- 푸틴 이어 김정은도 방중 가능성有 북한이 '전승절...,정지우 기자 (jjw@fnnews.com)
7,뉴스1,"윤 대통령 ""AI·디지털, 규제 정책이 중요…인류 후생 입각해 만들어야""",2023.09.13. 오후 12:35,"""가짜뉴스 확산하면 자유민주주의·시장경제·미래세대 망칠 수 있어""""정부, 초거대 A...",최동현 기자 (dongchoi89@news1.kr)
8,뉴스1,"한 총리, 체코 총리에 ""북한 오판않도록 강한 목소리 내달라""(종합)",2023.09.13. 오전 10:53,"체코 방문, 북러 회담 관련 당부…부산엑스포 유치 지지도K뷰티 박람회 방문…""더 열...",윤수희 기자 (ysh@news1.kr)
9,연합뉴스,"김기현, 국회서 교원단체와 간담회…교권회복 방안 논의",2023.09.13. 오전 5:00,국민의힘 김기현 대표[연합뉴스 자료사진](서울=연합뉴스) 김철선 기자 = 국민의힘 ...,김철선(kcs@yna.co.kr)


In [9]:
# 최종 결과 파일로 저장
all_sub_news_info_df.to_csv('./crawl_data/naver_sub_news_section.csv', index=0)

all_sub_news_info_df = pd.read_csv('./crawl_data/naver_sub_news_section.csv')
all_sub_news_info_df

,paper,title,datetime,writing,writer
0,SBS,"러시아 방문한 김정은 영상 공개돼…""내일 푸틴과 회담""",2023.09.12. 오후 11:44,4년 5개월 만에 러시아를 방문한 김정은 북한 국무위원장의 모습이 공개됐습니다.러시...,윤나라 기자 invictus@sbs.co.kr
1,파이낸셜뉴스,진중권 “박지현 눈물은 ‘공천용’...이재명에 회복식? 뻔한 거 아니냐”,2023.09.13. 오전 9:30,지난 11일 오후 단식 12일째인 이재명 더불어민주당 대표의 단식농성장을 찾은 박지...,박상훈 기자 (sanghoon3197@fnnews.com)
2,뉴시스,"김기현, 교원단체와 '교권회복' 간담회…법안 처리 고삐",2023.09.13. 오전 12:01,"교권회복 4법·아동학대처벌특례법 개정 등 논의여야, 교육위 법안소위서 교권회복 4법...",정성원 기자(jungsw@newsis.com)
3,데일리안,"단식 중인 이재명 항의방문했다고…민주당, 태영호 징계안 제출하기로",2023.09.12. 오전 11:37,"지난 7일 ""北에서 온 쓰레기"" 발언 항의방문송기헌 ""태영호 행태 도저히 묵과할 수...",김은지 기자 (kimeunji@dailian.co.kr)
4,조선비즈,"김기현, 자우림 김윤아 저격…“개념 없는 개념연예인”",2023.09.12. 오후 5:36,국민의힘 김기현 대표가 12일 오후 서울 중구 한국관광공사에서 열린 사단법인 문화자...,오귀환 기자 ogi@chosunbiz.com
5,KBS,"이재명 “윤석열 정부, 국방장관 교체로 꼬리자르기 열중…특검으로 진상 규명”",2023.09.13. 오전 10:04,더불어민주당 이재명 대표는 이종섭 국방부 장관의 사의 표명과 관련해 “윤석열 정부는...,고은희 (ginger@kbs.co.kr)
6,파이낸셜뉴스,‘한중일 대 북중러’ 대결 구도 굳어지나? 연쇄회담 가능성,2023.09.13. 오전 11:22,- 한중일에 맞서 결집하는 북러- 푸틴 이어 김정은도 방중 가능성有 북한이 '전승절...,정지우 기자 (jjw@fnnews.com)
7,뉴스1,"윤 대통령 ""AI·디지털, 규제 정책이 중요…인류 후생 입각해 만들어야""",2023.09.13. 오후 12:35,"""가짜뉴스 확산하면 자유민주주의·시장경제·미래세대 망칠 수 있어""""정부, 초거대 A...",최동현 기자 (dongchoi89@news1.kr)
8,뉴스1,"한 총리, 체코 총리에 ""북한 오판않도록 강한 목소리 내달라""(종합)",2023.09.13. 오전 10:53,"체코 방문, 북러 회담 관련 당부…부산엑스포 유치 지지도K뷰티 박람회 방문…""더 열...",윤수희 기자 (ysh@news1.kr)
9,연합뉴스,"김기현, 국회서 교원단체와 간담회…교권회복 방안 논의",2023.09.13. 오전 5:00,국민의힘 김기현 대표[연합뉴스 자료사진](서울=연합뉴스) 김철선 기자 = 국민의힘 ...,김철선(kcs@yna.co.kr)
